### Moirai

Github: https://github.com/SalesforceAIResearch/uni2ts

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/THESIS/rag-thesis'

/content/drive/MyDrive/THESIS/rag-thesis


In [ ]:
!pip install -U -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.6 MB/s eta 0:00:00
ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11; 1.6.2 Requires-Python >=3.7,<3.10; 1.6.3 Requires-Python >=3.7,<3.10; 1.7.0 Requires-Python >=3.7,<3.10; 1.7.1 Requires-Python >=3.7,<3.10; 1.7.2 Requires-Python >=3.7,<3.11; 1.7.3 Requires-Python >=3.7,<3.11; 1.8.0 Requires-Python >=3.8,<3.11; 1.8.0rc1 Requires-Python >=3.8,<3.11; 1.8.0rc2 Requires-Python >=3.8,<3.11; 1.8.0rc3 Requires-Python >=3.8,<3.11; 1.8.0rc4 Requires-Python >=3.8,<3.11; 1.8.1 Requires-Python >=3.8,<3.11
ERROR: Could not find a version that satisfies the requirement tdqdm (

In [ ]:
!git clone https://github.com/SalesforceAIResearch/uni2ts.git

Cloning into 'uni2ts'...
remote: Enumerating objects: 962, done.
remote: Counting objects: 100% (479/479), done.
remote: Compressing objects: 100% (206/206), done.
remote: Total 962 (delta 332), reused 276 (delta 273), pack-reused 483 (from 2)
Receiving objects: 100% (962/962), 8.26 MiB | 9.26 MiB/s, done.
Resolving deltas: 100% (467/467), done.
Updating files: 100% (252/252), done.


In [ ]:
cd '/content/drive/MyDrive/THESIS/rag-thesis/uni2ts'

/content/drive/MyDrive/THESIS/rag-thesis/uni2ts


In [ ]:
!pip install -e '.[notebook]'

Obtaining file:///content/drive/MyDrive/THESIS/rag-thesis/uni2ts
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for uni2ts (pyproject.toml) ... done
  Created wheel for uni2ts: filename=uni2ts-1.2.0-py3-none-any.whl size=14581 sha256=3d56fa35bec06b60fe6dd6648eb5597d890351abdee1626166dada966fa7a9e6
  Stored in directory: /tmp/pip-ephem-wheel-cache-o_59f3p3/wheels/55/c4/95/c866e0d1f00e3abd96c690f7848f144928bad20dd270475eb9
Successfully built uni2ts
  Attempting uninstall: uni2ts
    Found existing installation: uni2ts 1.2.0
    Uninstalling uni2ts-1.2.0:
      Successfully uninstalled uni2ts-1.2.0


In [ ]:
!touch .env

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
import torch
import matplotlib.pyplot as plt
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from huggingface_hub import hf_hub_download

from uni2ts.eval_util.plot import plot_single
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
from uni2ts.model.moirai_moe import MoiraiMoEForecast, MoiraiMoEModule

In [2]:
from statsmodels.tsa.ar_model import AutoReg
def run_ar_benchmark(train_data, test_data, target_var, ar_order=1, horizon=1):
    """
    Runs an AR(p) model on the transformed dataset and returns performance metrics.

    Args:
        train_data (pd.DataFrame): Training data
        test_data (pd.DataFrame): Test data
        target_var (str): Target variable name
        ar_order (int): Order of the AR model
        horizon (int): Number of future points to forecast
    """
    # Ensure the index has frequency information
    if not isinstance(train_data.index, pd.DatetimeIndex) or train_data.index.freq is None:
        train_data.index = pd.date_range(
            start=train_data.index[0],
            periods=len(train_data),
            freq='Q'
        )

    # Fit AR(p) model
    model = AutoReg(train_data[target_var], lags=ar_order)
    model_fit = model.fit()

    # Make predictions for the next h points
    predictions = model_fit.forecast(steps=horizon)

    # Get the actual values for the first h points of test data
    actuals = test_data[target_var][:horizon]

    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(actuals, predictions))
    return rmse

In [3]:
# Load dataframe
df = pd.read_csv('/content/drive/MyDrive/THESIS/rag-thesis/data/processed_FRED_data.csv', index_col=0, parse_dates=True)
df.index = pd.to_datetime(df.index)
variables = list(df.select_dtypes(include=[np.number]).columns)
df

,GDP_log,XTIMVA01USM664S_log,XTEXVA01USM664S_log,GPDI_log,RPI_log,INDPRO_log,BUSLOANS_log,NONREVSL_log,UNRATE_diff,DFF_diff,GS1_diff,AAA_diff,IRLTLT01USM156N_diff,M1V,CUMFNS,CLF16OV,B235RC1Q027SBEA,ROWFDIQ027S,CPILFESL_inflation,PCEPI_inflation
observation_date,,,,,,,,,,,,,,,,,,,,
1960-04-01,-1.889106,-1.937830,-1.900735,-1.933337,-1.969102,-2.223121,-1.891318,-1.855541,0.143477,-0.270617,-0.991366,-0.273408,-0.492671,-1.208557,0.410447,-1.730502,-0.917074,-0.751348,-1.084373,-0.461731
1960-07-01,-1.882088,-1.966669,-1.884644,-1.940230,-1.964744,-2.260716,-1.847751,-1.841514,0.420758,-0.880542,-1.133084,-0.381349,-0.925014,-1.210446,-0.080174,-1.724082,-0.920047,-0.750873,-1.585535,-0.688792
1960-10-01,-1.890482,-2.003349,-1.890457,-2.047059,-1.966695,-2.316039,-1.805262,-1.830178,1.113960,-0.742446,-0.112715,0.023429,0.134228,-1.227453,-0.698059,-1.713532,-0.921271,-0.752034,-0.573631,-0.579965
1961-01-01,-1.882994,-2.008331,-1.877366,-2.021036,-1.951916,-2.350629,-1.773634,-1.821522,0.698039,-0.362681,-0.169402,-0.138482,-0.211647,-1.220367,-1.108528,-1.704369,-0.921796,-0.750767,-1.086546,-0.984947
1961-04-01,-1.866870,-2.003468,-1.911936,-1.962365,-1.926400,-2.262478,-1.750164,-1.825352,0.282117,-0.305141,0.099862,0.023429,0.004525,-1.198636,-0.596247,-1.704078,-0.922670,-0.750318,-0.916503,-1.296390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-01,1.436028,1.213070,1.285307,1.444928,1.501814,1.137583,1.229209,1.545975,0.282117,0.304784,0.638390,0.832985,1.215087,-2.314943,-0.246743,1.425838,2.504592,0.907744,-0.275020,-0.228830
2023-10-01,1.445893,1.216451,1.284562,1.451383,1.515121,1.127452,1.229436,1.545675,0.143477,0.074624,-0.240261,0.806000,0.631423,-2.296519,-0.351921,1.432710,2.512636,0.775326,-0.134930,-0.631986
2024-01-01,1.455579,1.227645,1.290731,1.460634,1.539558,1.117367,1.236763,1.547141,0.004836,-0.005932,-0.452838,-0.651201,-0.600757,-2.286126,-0.435661,1.426999,2.630025,0.969136,0.193151,0.053674


In [4]:
def run_moirai(df, ratio=0.7):
    variables = list(df.select_dtypes(include=[np.number]).columns)
    # Horizons to evaluate
    horizons = [1,3,6,12]
    results = {
        h: {
            'RMSE': [],
            'cum_RMSE': {var:[] for var in variables}
        } for h in ["h=1", "h=3", "h=6", "h=12"]
    }

    for h in horizons:
        print(f"Running for horizon {h}...")
        MODEL = "moirai-moe"  # model name: choose from {'moirai', 'moirai-moe'}
        SIZE = "small"  # model size: choose from {'small', 'base', 'large'}
        PDT = h  # prediction length: any positive integer
        CTX = 200  # context length: any positive integer
        PSZ = "auto"  # patch size: choose from {"auto", 8, 16, 32, 64, 128}
        BSZ = 32  # batch size: any positive integer
        TEST = 79  # test set length: any positive integer

        # Convert into GluonTS dataset
        ds = PandasDataset(dict(df),freq='Q')

        # Split into train/test set
        train, test_template = split(
            ds, offset=-TEST
        )  # assign last TEST time steps as test set

        # Construct rolling window evaluation
        test_data = test_template.generate_instances(
            prediction_length=PDT,  # number of time steps for each prediction
            windows=60,  # number of windows in rolling window evaluation
            distance=1,  # number of time steps between each window - distance=PDT for non-overlapping windows
        )

        # Prepare model
        if MODEL == "moirai":
            model = MoiraiForecast(
                module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.1-R-small"),
                prediction_length=PDT,
                context_length=CTX,
                patch_size=PSZ,
                num_samples=100,
                target_dim=1,
                feat_dynamic_real_dim=ds.num_feat_dynamic_real,
                past_feat_dynamic_real_dim=ds.num_past_feat_dynamic_real,
            )
        elif MODEL == "moirai-moe":
            model = MoiraiMoEForecast(
                module=MoiraiMoEModule.from_pretrained(f"Salesforce/moirai-moe-1.0-R-small"),
                prediction_length=PDT,
                context_length=CTX,
                patch_size=16,
                num_samples=100,
                target_dim=1,
                feat_dynamic_real_dim=ds.num_feat_dynamic_real,
                past_feat_dynamic_real_dim=ds.num_past_feat_dynamic_real,
            )

        predictor = model.create_predictor(batch_size=BSZ)
        forecasts = predictor.predict(test_data.input)

        input_it = iter(test_data.input)
        label_it = iter(test_data.label)
        forecast_it = iter(forecasts)

        # Test train split for AR data (wide format)
        split_idx = 180
        df_ar = df.copy()
        train_data_ar = df_ar.iloc[:split_idx]
        test_data_ar = df_ar.iloc[split_idx:]

        # Calculate RMSE and statistics for each variable
        current_cum_sums ={var:0 for var in variables}
        rmses = {var:[] for var in variables}  # Store all RMSE values for this horizon
        ar_rmses = {var:[] for var in variables}

        # Add rolling window RMSEs to rmses dict for each variable
        for var in variables:
            for i in range(60):
                label = next(label_it)
                forecast = next(forecast_it)
                rmse = np.sqrt(mean_squared_error(forecast.mean, label['target']))
                rmses[var].append(rmse)

            for val in rmses[var]:
                current_cum_sums[var] += val
                results["h="+str(h)]['cum_RMSE'][var].append(current_cum_sums[var])

        while split_idx < 245:
            # Calculate metrics for each unique_id (variable)
            for var in variables:
                ar1_rmse = run_ar_benchmark(train_data_ar, test_data_ar, var, ar_order=1, horizon=h)
                ar_rmses[var].append(ar1_rmse)

            # Shift test-train split by 1 timestamp
            split_idx += 1
            train_data_ar = df_ar.iloc[:split_idx]
            test_data_ar = df_ar.iloc[split_idx:]

        for var in variables:
            # Compute root mean square error for each variable
            results["h="+str(h)]['RMSE'].append(np.sum(rmses[var])/np.sum(ar_rmses[var]))

    return results


moirai_results = run_moirai(df)

# Print results
print("\nDetailed RMSE for each variable and horizon:")
print(moirai_results)


Running for horizon 1...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Running for horizon 3...
Running for horizon 6...
Running for horizon 12...

Detailed RMSE for each variable and horizon:
{'h=1': {'RMSE': [0.8240613613058143, 0.7706002243906542, 0.7021689930069128, 0.827807104075522, 0.6343739098928687, 0.6198902339059895, 0.6516816072542074, 0.9979312384980981, 0.2969999755669151, 0.7329371334238531, 0.7999758510595069, 0.9946757780054597, 0.958654752050657, 0.49185570726293665, 0.5674513568260001, 0.6838905980252705, 0.647479948841776, 0.8073690849386033, 0.5493263271402402, 0.7978009050524612], 'cum_RMSE': {'GDP_log': [0.006364732386156646, 0.010759231398897229, 0.016821038944321187, 0.023802669416106093, 0.03445934063958489, 0.03807792439191138, 0.03975569124483913, 0.04463905492993636, 0.04799757319156994, 0.053634545752041074, 0.0540414005195482, 0.05543206894611963, 0.061752306341538055, 0.06185061575829742, 0.06648057519288209, 0.09401136012287215, 0.10445436851301271, 0.11471352433984727, 0.11805166832350955, 0.1342012458335401, 0.1359807291

In [5]:
for h in moirai_results:
    print(f"Horizon {h}:")
    rmse_array = np.array(moirai_results[h]['RMSE'])
    #rmse_array = rmse_array[rmse_array <= 15]
    # Calculate statistics for this horizon
    print("Mean:", np.mean(rmse_array))
    print("Std: ", np.std(rmse_array))
    print("Min: ", np.min(rmse_array))
    print("Max: ", np.max(rmse_array))
    print("Q25: ", np.percentile(rmse_array, 25))
    print("Median: ", np.percentile(rmse_array, 50))
    print("Q75: ", np.percentile(rmse_array, 75))
    print("IQR: ", np.percentile(rmse_array, 75) - np.percentile(rmse_array, 25))

Horizon h=1:
Mean: 0.7178466045261873
Std:  0.16838926437469065
Min:  0.2969999755669151
Max:  0.9979312384980981
Q25:  0.6307529908961489
Median:  0.717553063215383
Q75:  0.811542154030406
IQR:  0.18078916313425708
Horizon h=3:
Mean: 0.7741410014115597
Std:  0.1415377837616259
Min:  0.45979713559306074
Max:  1.0119944243657972
Q25:  0.7052283760717897
Median:  0.763588208311683
Q75:  0.878835686369368
IQR:  0.17360731029757837
Horizon h=6:
Mean: 0.8339005506347466
Std:  0.12686513362976592
Min:  0.5322038960870884
Max:  1.0312915790158548
Q25:  0.7514425368713243
Median:  0.8367144800262725
Q75:  0.9437471339428525
IQR:  0.19230459707152814
Horizon h=12:
Mean: 0.9039179407818454
Std:  0.13620480481408112
Min:  0.554092566719164
Max:  1.164442152795178
Q25:  0.8293809166805108
Median:  0.9304412879602388
Q75:  0.99247371718299
IQR:  0.1630928005024792


In [ ]:
import json
with open('../results/moirai_results.json', 'w') as f:
    json.dump(moirai_results, f,indent=4,)